In [6]:
import scipy
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
from scipy import stats
from scipy.stats import binom, poisson, norm           # Functions from SciPy Stats...
# import scipy.optimize as optimization
from scipy import optimize
from numpy.linalg import inv
from functools import partial
from fitter import Fitter
from astropy.modeling import models, fitting
from scipy.optimize import leastsq
import math

In [7]:
sys.path.append('D:\my github\Siyi Applied Stats\Documents for JN\AppStat2021-main\External_Functions\\')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax    # Useful functions to print fit results on figure

plt.rcParams['font.size'] = 18     # Set some basic plotting parameters

### 因为重复运用变量名，每次运行时须：中断内核-清空所有结果-rerun

### T-test

In [ ]:
# one-sample test, dof:n-1
mean1=550.75   # sample mean
s1=4.25   # uncertainty
n1=16
mean0=550  # compared mean
t= (mean1-mean0)/(s1/np.sqrt(n1))
dof= n1-1
print(t)
print(dof)

# two-sample test with same variance,dof: n-2
mean1=14.95
s1= 6.84   # uncertainty1
n1=13
mean2=22.29
s2=5.32   # uncertainty2
n2=10
sp= np.sqrt(((n1-1)*s1**2+(n2-1)*s2**2)/(n1+n2-2))  # combined estimate variance
t= (mean1-mean2)/np.sqrt(sp**2/n1 + sp**2/n2)
dof= n1+n2-2
print(t)
print(dof)

# two-sample test with different variance,dof: Welch-Satterhwaite equation
mean1=0.0239   
s1=0.0005   # uncertainty1
n1=103261
mean2=0.0188
s2=0.0008   # uncertainty2
n2=26162
t= (mean1-mean2)/np.sqrt(s1**2/n1 + s2**2/n2)
dof= (s1**2/n1 + s2**2/n2)**2/((s1**2/n1)**2/(n1-1)+(s2**2/n2)**2/(n2-1))
print(t)
print(dof)

# paired test
x1 = [63,65,56,100,88,83,77,92,90,84,68,74,87,64,71,88]
x2 = [69,65,62,91,78,87,79,88,85,92,69,81,84,75,84,82]
n = 16  # number of x1=x2
mean0= 0  # compared mean
d = np.zeros(n)  # d=x1-x2
for i in range(n):
    d[i] = x1[i]-x2[i]
mean_d = np.mean(d)
s_d = np.std(d,ddof=1)  # if std_sample, add",ddof=1".if std_population,delete ",ddof=1"
print(d)
print(mean_d,s_d)

t= (mean_d-mean0)/(s_d/np.sqrt(n))
dof= n-1
print(t)
print(dof)

### F-test

In [5]:
# f-test
s1_square = 31  # variance1
n1=11  # number of sample1
s2_square = 20  # variance2
n2=21  # number of sample2
if s1_square > s2_square:
    f = s1_square/s2_square
else: 
    f = s2_square/s1_square
df1=n1-1
df2=n2-1
print(f)
print(df1,df2)

1.55
10 20


### Chi-square test

In [ ]:
# Chi2 test
x = [9.54,9.36,10.02,9.87,9.98,9.86,9.86,9.81,9.79]  # x is observed value
y = [0.15,0.10,0.11,0.08,0.14,0.06,0.03,0.13,0.04]  # y is observed sigma

x1 = np.ones_like(x)*9.824  # x1 is estimated value
#y1 = np.ones_like(x)*0.02  # y1 is estimated sigma

np.set_printoptions(suppress = True)
Npoints = 9  # number of samples
sum1 = np.zeros_like(x)
chi2 = np.zeros_like(x)
sigma2 = np.zeros_like(x)

for i in range(0,Npoints):
    sum1[i] = (x[i]-x1[i])**2  # sum1 is (observed value-estimated value)**2
    
for k in range(0,Npoints):
    sigma2[k] = (y[k])**2   # sigma2 is (observed sigma)**2
    
for j in range(0,Npoints):
    chi2[j] += sum1[j]/sigma2[j]  # Chi2

Chi2_fit = np.sum(chi2)  # calculate Chi2

# calculate probability
Nvar = 1  # number of fit parameter
Ndof_fit = Npoints - Nvar
Prob_fit = stats.chi2.sf(Chi2_fit,Ndof_fit)
print("Chi2:",Chi2_fit)
print("nDOF:",Ndof_fit)
print("p-value:",Prob_fit)

### Kolmogorov-Smirnov/K-S test 
以下三种检验需要是连续分布函数。
K-S检验统计量用的是经验累积概率与目标理论累积概率之差的最大值，这有点像计算极差。

In [54]:
# K-S test, one-sample
from scipy.stats import kstest
import numpy as np

x = np.random.normal(0,1,1000)
test_stat = kstest(x, 'norm')  # or "expon"
print(test_stat)

# K-S test, two-sample
from scipy.stats import ks_2samp
import numpy as np

beta=np.random.beta(7,5,1000)  # distribution1
norm=np.random.normal(0,1,1000)  # distribution2
ks_value=ks_2samp(beta,norm)
print(ks_value)


KstestResult(statistic=0.01921202617523099, pvalue=0.8470246667462732)
KstestResult(statistic=0.622, pvalue=2.5872247709377997e-181)


### Anderson-Darling test
AD检验则把所有的差平方后求和，有点像计算方差。当然计算方差的稳健性(Robustness)要好一些，这可能也是AD检验得到更多应用的原因。

In [58]:
from scipy.stats import anderson
import numpy as np

x = np.random.normal(0,1,1000)
test_stat = anderson(x, 'norm')  # or "expon","logistic","gumbel"
print(test_stat)  # if statistic value > critical value, then could reject

AndersonResult(statistic=0.2678463840571794, critical_values=array([0.574, 0.653, 0.784, 0.914, 1.088]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))


### Shapiro-Wilks test
SW检验更像是计算相关系数，这个数越接近1，则越有可能正态。

In [64]:
from scipy.stats import shapiro
data = [0.86, 0.78, 0.83, 0.84, 0.77, 0.84, 0.81, 0.84, 0.81, 0.81, 0.80, 0.81,
       0.79, 0.74, 0.82, 0.78, 0.82, 0.78, 0.81, 0.80, 0.81, 0.74, 0.87, 0.78]
stat, p = shapiro(data)
print("stats-value is ：%f" %stat,"p-value is：%f" %p)

stats-value is ：0.966175 p-value is：0.574134


### Wald-Wolfowitz runs test
requires N>10-15. If not, use equations on textbook p154

In [71]:
N_a= 6  # runs of a
N_b=6  # runs of b
N = N_a+N_b  # sum run
mean = 1+ (2*N_a*N_b)/N
variance = (2*N_a*N_b*(2*N_a*N_b-N))/(N**2*(N-1))
sigma = np.sqrt(variance)
print("mean is：%f" %mean,"sigma is：%f" %sigma)

mean is：7.000000 sigma is：1.651446


然后计算mean偏离标准mean多少个sigma,即得出z值。通过查正态分布表得出是否拒绝零假设的结论。

### 生成t分布

In [ ]:
stats.t.rvs(3,size=100) # t-distribution,df=3
stats.t.rvs(100,size=100) # t-distribution,df=100,similar to Gaussian distribution

### Fisher's exact test

In [9]:
# Fisher's exact test
A= 1  # row1,column1
B= 9  # row2,column1
C= 11  # row1,column2
D= 3  # row2,column2

def main(num):
    return math.factorial(num)
p= (main(A+B)*main(C+D)*main(A+C)*main(B+D))/(main(A)*main(B)*main(C)*main(D)*main(A+B+C+D))

print(p)

0.0013460761879122358


### Fit function

In [ ]:
# Gaussian distribution


# 4-parameter function, s-shape curve
def fourPL(x, A, B, C, D):
    return ((A-D)/(1.0+((x/C)**(B))) + D)
        
# log-normal distribution
def fit_pdf(x, a, mu, sigma):
    return a / x * 1. / (sigma * np.sqrt( 2. * np.pi ) ) * np.exp( -( np.log( x ) - mu )**2 / ( 2. * sigma**2 ) )

# polynomial 
def fit_pdf(x,A,B,C,D,E):
    return A*x**4 + B*x**3 + C*x**2 + D*x*1 + E

# exponential+Gaussian distribution
def fit_pdf(x, Nexp, tau, Ngauss, mu, sigma) :
    return Nexp * binwidth * (1.0 / tau * np.exp(-x/tau))+ Ngauss * binwidth * (1.0 / np.sqrt(2*np.pi) / sigma * np.exp( -0.5 * (x-mu)**2 / sigma**2))

# Gamma distribution (sum of exponential distributions)
def fit_pdf(x,a,b,c):
    return a*(x**b)*np.exp(-c*x)   

In [ ]:
from pynverse import inversefunc
#它可用于计算某些y_values 点的反函数：
cube = (lambda x: np.sqrt(x**4))
invcube = inversefunc(cube, y_values=16)
print(invcube)

### Plot: axis log-log

In [ ]:
# axis log-log
ax.set_yscale('log',nonposy='mask',subsy=[0]) # y-axis log
plt.loglog(x,y, 'r-', Marker='.') # x、y-axis log

### Misc

In [48]:
# 意为开a的r次方
math.pow(a,1/r) 

983.6104699753055
31517.23052510799
